In [107]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd

size = 250
rsi_window = 14
roc_window = 5
volatility_window = 30

In [ ]:
from yfinance import EquityQuery, screen
import pandas as pd

# Define your screen query (as you already have)
screen_query = EquityQuery("and", [
    EquityQuery("is-in", ["region", "us"]),
    EquityQuery("is-in", ["sector",
        "Technology",
        "Financial Services",
        "Consumer Cyclical",
        "Communication Services",
        "Basic Materials",
        "Industrials"
    ]),
    EquityQuery("GTE", ["eodprice", 0.50]),
    EquityQuery("GTE", ["avgdailyvol3m", 100000]),
    EquityQuery("LTE", ["short_percentage_of_float.value", 100])
])

batch_size = 250
offset = 0
all_stocks = []

while True:
    screener = screen(
        screen_query,
        sortField="avgdailyvol3m",
        sortAsc=False,
        size=batch_size,
        offset=offset
    )
    stocks = screener.get('quotes', [])
    if not stocks:
        break
    all_stocks.extend(stocks)
    offset += batch_size
    print(f"Loaded {len(all_stocks)} stocks")

# Convert to DataFrame
stocks_df = pd.DataFrame(all_stocks)
print(f"Total stocks loaded: {len(stocks_df)}")



Loaded 250 stocks so far...
Loaded 500 stocks so far...
Loaded 750 stocks so far...
Loaded 1000 stocks so far...
Loaded 1250 stocks so far...
Loaded 1500 stocks so far...
Loaded 1750 stocks so far...
Loaded 2000 stocks so far...
Loaded 2050 stocks so far...
Total stocks loaded: 2050


In [109]:
stocks = pd.DataFrame(stocks)
stocks = stocks.drop(columns=[
    'esgPopulated',
    'cryptoTradeable',
    'hasPrePostMarketData',
    'firstTradeDateMilliseconds',
    'priceHint',
    'regularMarketDayHigh',
    'regularMarketDayRange',
    'regularMarketDayLow',
    'regularMarketPreviousClose',
    'messageBoardId',
    'fullExchangeName',
    'financialCurrency',
    'regularMarketOpen',
    'averageDailyVolume3Month',
    'averageDailyVolume10Day',
    'corporateActions',
    'fiftyTwoWeekLowChange',
    'fiftyTwoWeekLowChangePercent',
    'fiftyTwoWeekRange',
    'fiftyTwoWeekHighChange',
    'fiftyTwoWeekHighChangePercent',
    'fiftyTwoWeekChangePercent',
    'earningsTimestamp',
    'earningsTimestampStart',
    'earningsTimestampEnd',
    'earningsCallTimestampStart',
    'earningsCallTimestampEnd',
    'dividendRate',
    'trailingAnnualDividendYield',
    'marketState',
    'epsCurrentYear',
    'fiftyDayAverage',
    'fiftyDayAverageChange',
    'fiftyDayAverageChangePercent',
    'twoHundredDayAverage',
    'twoHundredDayAverageChange',
    'twoHundredDayAverageChangePercent','dividendYield','gmtOffSetMilliseconds','bid','ask','currency', 'quoteType','language','typeDisp','quoteSourceName','triggerable','customPriceAlertConfidence','sourceInterval','exchangeDataDelayedBy','exchangeTimezoneName','exchangeTimezoneShortName',
 'bidSize','askSize','market','isEarningsDateEstimate','trailingAnnualDividendRate','regularMarketTime','priceEpsCurrentYear','epsForward','forwardPE','tradeable','displayName','longName'
]
)
df = stocks

In [110]:
order = [
    # Identification
    'symbol',
    'shortName',
    'exchange',
    'region',
    
    # Market Price & Change
    'regularMarketPrice',
    'regularMarketChange',
    'regularMarketChangePercent',
    'regularMarketVolume',
    
    # Valuation Metrics
    'marketCap',
    'sharesOutstanding',
    'bookValue',
    'priceToBook',
    'trailingPE',
    'epsTrailingTwelveMonths',
    'averageAnalystRating',
    
    # Price Range
    'fiftyTwoWeekHigh',
    'fiftyTwoWeekLow',
    
    # Dividend Info
    'dividendDate'
]
df = df[order]
df


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate
0,NVDA,NVIDIA Corporation,NMS,US,135.13,-4.060000,-2.916870,323313507,3394506063872,24387600384,3.438,39.304830,44.893690,3.01,1.4 - Strong Buy,153.13,86.620,1.743552e+09
1,F,Ford Motor Company,NYQ,US,10.38,0.160000,1.565560,132007518,40640294912,3905700096,11.225,0.924722,8.173228,1.27,3.0 - Hold,14.85,8.440,1.748822e+09
2,TSLA,"Tesla, Inc.",NMS,US,346.46,-11.970000,-3.339560,122775188,1154487812096,3220960000,23.184,14.943927,203.799990,1.70,2.6 - Hold,488.54,167.410,NaN
3,LCID,"Lucid Group, Inc.",NMS,US,2.23,-0.180000,-7.468880,254714279,7351128064,3050259968,1.044,2.136015,NaN,-1.10,3.1 - Hold,4.43,1.930,NaN
4,PLTR,Palantir Technologies Inc.,NMS,US,131.78,9.460000,7.733810,182153344,298206265344,2262909952,2.299,57.320570,527.120000,0.25,3.1 - Hold,133.49,21.005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,SYF,Synchrony Financial,NYQ,US,57.65,-0.599998,-1.030040,5392569,22173286400,380656992,40.361,1.428359,7.984765,7.22,2.0 - Buy,70.93,40.545,1.747267e+09
246,ZETA,Zeta Global Holdings Corp.,NYQ,US,13.14,0.050000,0.381972,4402454,3095048192,211448992,2.866,4.584787,NaN,-0.25,NaN,38.20,10.690,NaN
247,JCI,Johnson Controls International,NYQ,US,101.37,-0.769997,-0.753864,6829361,67212976128,658046976,24.018,4.220585,30.905489,3.28,2.0 - Buy,102.36,64.310,1.744848e+09
248,ETSY,"Etsy, Inc.",NMS,US,55.35,2.360000,4.453660,7094382,5772008448,104282000,-8.659,-6.392193,38.437496,1.44,2.7 - Hold,66.93,40.050,NaN


In [111]:
# Get the price history for the stocks, to do RSI, or MACD, options volatility, and kalman filter acceleration(or see tview workaround)
stocklist = df.symbol.tolist()
stocklist

# Get the price history for the stocks
price_data = yf.download(stocklist, period='14d', interval='1d')
price_data = price_data['Close']

# Calculate the daily returns
change_data = price_data[stocklist]/price_data[stocklist].shift(1)-1

# RSI
change_data
def rsi(price_data, period=rsi_window):
    delta = change_data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
rsi_data = rsi(price_data)
rsi_data

rsi = round(rsi_data.iloc[-1],1)
df['RSI'] = rsi.values


## Rate of change 
window=5 
recent_avg = change_data.rolling(window=roc_window).mean()
earlier_avg = change_data.shift(window).rolling(window=roc_window).mean()

# Acceleration: difference between recent and earlier averages
accel = (recent_avg - earlier_avg) * 100
accel = round(accel.iloc[-1], 2)
df['ROC'] = accel.values


# Apply filters

filterdf = df[(df['RSI'] < 60) ] # Filter for RSI < 60

filterdf



[*********************100%***********************]  250 of 250 completed
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_41866/2194723550.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = rsi.values
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_41866/2194723550.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ROC'] = accel.values


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate,RSI,ROC
0,NVDA,NVIDIA Corporation,NMS,US,135.13,-4.060000,-2.916870,323313507,3394506063872,24387600384,3.438,39.304830,44.893690,3.01,1.4 - Strong Buy,153.13,86.620,1.743552e+09,36.6,0.67
1,F,Ford Motor Company,NYQ,US,10.38,0.160000,1.565560,132007518,40640294912,3905700096,11.225,0.924722,8.173228,1.27,3.0 - Hold,14.85,8.440,1.748822e+09,55.3,0.47
2,TSLA,"Tesla, Inc.",NMS,US,346.46,-11.970000,-3.339560,122775188,1154487812096,3220960000,23.184,14.943927,203.799990,1.70,2.6 - Hold,488.54,167.410,NaN,41.6,0.46
3,LCID,"Lucid Group, Inc.",NMS,US,2.23,-0.180000,-7.468880,254714279,7351128064,3050259968,1.044,2.136015,NaN,-1.10,3.1 - Hold,4.43,1.930,NaN,44.9,-3.37
4,PLTR,Palantir Technologies Inc.,NMS,US,131.78,9.460000,7.733810,182153344,298206265344,2262909952,2.299,57.320570,527.120000,0.25,3.1 - Hold,133.49,21.005,NaN,49.5,2.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,AI,"C3.ai, Inc.",NYQ,US,26.59,-1.210000,-4.352510,11259516,3690133760,129239000,6.292,4.226001,NaN,-2.14,3.1 - Hold,45.08,17.030,NaN,48.3,4.09
245,SYF,Synchrony Financial,NYQ,US,57.65,-0.599998,-1.030040,5392569,22173286400,380656992,40.361,1.428359,7.984765,7.22,2.0 - Buy,70.93,40.545,1.747267e+09,45.5,1.09
246,ZETA,Zeta Global Holdings Corp.,NYQ,US,13.14,0.050000,0.381972,4402454,3095048192,211448992,2.866,4.584787,NaN,-0.25,NaN,38.20,10.690,NaN,45.3,0.79
247,JCI,Johnson Controls International,NYQ,US,101.37,-0.769997,-0.753864,6829361,67212976128,658046976,24.018,4.220585,30.905489,3.28,2.0 - Buy,102.36,64.310,1.744848e+09,43.5,0.46


#"""Steps:
#1. get options volatility, prices, data 
#2. get RSI, kalman filter acceleration  (done)
#indicators in screener: filter by RSI, filter by relative options volatility (vs historical volatility, as yfinance does not have historical options volatility data)
#"""
# other:
#take steps to pull all yahoo screen companies 

In [ ]:
filterstocks = filterdf.symbol.tolist()
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')["Close"]
#^^^ fix 
# Historical volatiltiy calculation
# Next add the vol smile relative to HV 
def hv(price_datayear, window=volatility_window):
    log_returns = np.log(price_datayear / price_datayear.shift(1))
    volatility = log_returns.rolling(window=window).std() * np.sqrt(252)  # Annualized volatility
    return volatility

hvol = hv(price_datayear)
hvol

[*********************100%***********************]  244 of 244 completed
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


Price          Close                                                    \
Ticker            AA       AAL      AAPL      ABNB       ADT       AEG   
Date                                                                     
2025-02-28       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-03       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-04       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-05       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-06       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23  0.506467  0.417077  0.356372  0.329126  0.236214  0.184908   
2025-05-27  0.495151  0.427492  0.344716  0.332934  0.232980  0.178736   
2025-05-28  0.497425  0.431043  0.338702  0.330947  0.219191  0.176354   
2025-05-29  0.496307  0.426602  0.338724  0.330553  0.219353  0.178123   
2025-05-30  0.505383  0.400287  0.317624  0.324288  0.212495  0.173690   

Price                                               ...    Volume            \
Ticker           AEO      AFRM      AGMH        AI  ...       WFC       WIT   
Date                                                ...                       
2025-02-28       NaN       NaN       NaN       NaN  ...       NaN       NaN   
2025-03-03       NaN       NaN       NaN       NaN  ...       NaN       NaN   
2025-03-04       NaN       NaN       NaN       NaN  ...       NaN       NaN   
2025-03-05       NaN       NaN       NaN       NaN  ...       NaN       NaN   
2025-03-06       NaN       NaN       NaN       NaN  ...       NaN       NaN   
...              ...       ...       ...       ...  ...       ...       ...   
2025-05-23  0.524299  0.843214  4.273777  0.458330  ...  4.306104  6.758272   
2025-05-27  0.534388  0.847045  4.273939  0.478019  ...  4.313877  6.970234   
2025-05-28  0.534429  0.848586  4.270426  0.486206  ...  4.330033  7.454987   
2025-05-29  0.527104  0.849720  4.274245  0.720651  ...  4.507275  7.544731   
2025-05-30  0.530477  0.846306  4.282991  0.728243  ...  4.515519  7.539085   

Price                                                                   \
Ticker            WU         X        XP      XPEV       XRX       YMM   
Date                                                                     
2025-02-28       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-03       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-04       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-05       NaN       NaN       NaN       NaN       NaN       NaN   
2025-03-06       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23  4.820176  7.884695  7.556079  8.050006  7.114829  7.988380   
2025-05-27  4.817514  7.890964  7.474222  8.044160  7.323320  7.759553   
2025-05-28  4.752828  8.270007  7.413712  7.993083  7.302856  7.760937   
2025-05-29  4.585103  8.395089  7.452128  7.962304  7.330386  7.448501   
2025-05-30  4.674023  8.784511  7.546262  7.961581  7.324148  7.470957   

Price                           
Ticker          ZETA       ZIM  
Date                            
2025-02-28       NaN       NaN  
2025-03-03       NaN       NaN  
2025-03-04       NaN       NaN  
2025-03-05       NaN       NaN  
2025-03-06       NaN       NaN  
...              ...       ...  
2025-05-23  6.773606  9.623392  
2025-05-27  6.747092  9.645876  
2025-05-28  6.769176  9.700479  
2025-05-29  6.783142  9.700968  
2025-05-30  6.693784  9.663818  

[64 rows x 1220 columns]

In [113]:
filterstocks = filterdf.symbol.tolist()
#  1. Download price data (make sure columns are tickers)
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')['Close']

# 2. Calculate HV
hvol_latest = price_datayear.apply(hv).iloc[-1]
hvol_latest.name = 'hvol'

# 3. Prepare for merge
filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()
hvol_latest.index = hvol_latest.index.astype(str).str.upper()

# 4. Merge
filterdf = filterdf.merge(hvol_latest, left_on='symbol', right_index=True, how='left')


[*********************100%***********************]  244 of 244 completed
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_41866/1090464252.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()


In [114]:
import yfinance as yf
import pandas as pd

def find_closest_value(data_list, target_value):
    """Find the value in a list or Series that is closest to a given target value."""
    if len(data_list) == 0:
        return None
    return min(data_list, key=lambda x: abs(x - target_value))

def filter_special_expiry(expiry_list):
    """
    Returns a list of expiries filtered to only the 2nd lowest, middle, 3rd to last, and last.
    """
    sorted_expiries = sorted(expiry_list)
    length = len(sorted_expiries)
    if length == 0:
        return []
    values_to_keep = set()
    if length > 1:
        values_to_keep.add(sorted_expiries[1])  # 2nd lowest
    values_to_keep.add(sorted_expiries[length // 2])  # middle
    if length >= 3:
        values_to_keep.add(sorted_expiries[-3])  # 3rd to last
    values_to_keep.add(sorted_expiries[-1])  # last
    return list(values_to_keep)

def get_options(symbol):
    try:
        stock = yf.Ticker(symbol)
        options_chain = stock.options
        if not options_chain:
            return pd.DataFrame()
        # Filter expiries first
        filtered_expiries = filter_special_expiry(options_chain)
        options_data = []
        current_price = stock.history(period='1d')['Close'].iloc[-1]
        for expiry in filtered_expiries:
            chain = stock.option_chain(expiry)
            for opt_type, df in [('call', chain.calls), ('put', chain.puts)]:
                if not df.empty:
                    closest_strike = find_closest_value(df['strike'], current_price)
                    filtered = df[df['strike'] == closest_strike].copy()
                    filtered['type'] = opt_type
                    filtered['expiry'] = expiry
                    filtered['symbol'] = symbol
                    options_data.append(filtered)
        if options_data:
            return pd.concat(options_data, ignore_index=True)
        else:
            return pd.DataFrame()
    except Exception as e:
        print(f"Error fetching options for {symbol}: {e}")
        return pd.DataFrame()

# Example usage:
# filterstocks = ['AAPL', 'MSFT', 'NVDA', ...]
options_list = [get_options(symbol) for symbol in filterstocks] ### Change this to filterstocks to get all options

options = pd.concat(options_list, ignore_index=True)
options = options[options["openInterest"]>100]
options

$AZUL: possibly delisted; no price data found  (period=1d)


Error fetching options for AZUL: single positional indexer is out-of-bounds


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,type,expiry,symbol
0,NVDA250613C00135000,2025-05-30 19:59:57+00:00,135.0,4.15,4.10,4.15,-2.950000,-41.549294,7791.0,7038.0,0.387213,True,REGULAR,USD,call,2025-06-13,NVDA
1,NVDA250613P00135000,2025-05-30 19:59:59+00:00,135.0,4.00,4.00,4.05,1.280000,47.058823,5234.0,3571.0,0.390021,False,REGULAR,USD,put,2025-06-13,NVDA
2,NVDA271217C00135000,2025-05-30 19:58:41+00:00,135.0,44.60,44.05,44.60,-2.600002,-5.508480,1306.0,5449.0,0.529546,True,REGULAR,USD,call,2027-12-17,NVDA
3,NVDA271217P00135000,2025-05-30 19:33:27+00:00,135.0,32.02,31.85,32.35,1.080000,3.490627,67.0,1873.0,0.382788,False,REGULAR,USD,put,2027-12-17,NVDA
4,NVDA251121C00135000,2025-05-30 19:58:37+00:00,135.0,18.05,17.40,17.90,-2.400002,-11.735949,1203.0,5561.0,0.480169,True,REGULAR,USD,call,2025-11-21,NVDA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761,IBM250613C00260000,2025-05-30 19:24:08+00:00,260.0,4.67,3.05,6.25,-0.150000,-3.112035,236.0,165.0,0.330939,False,REGULAR,USD,call,2025-06-13,IBM
1763,IBM270115C00260000,2025-05-29 19:17:56+00:00,260.0,39.66,38.85,40.75,0.000000,0.000000,2.0,537.0,0.313758,False,REGULAR,USD,call,2027-01-15,IBM
1764,IBM270115P00260000,2025-05-29 17:48:24+00:00,260.0,33.80,31.60,36.25,0.000000,0.000000,87.0,1095.0,0.272194,True,REGULAR,USD,put,2027-01-15,IBM
1765,IBM260618C00260000,2025-05-28 19:48:20+00:00,260.0,33.65,31.50,32.65,0.000000,0.000000,10.0,181.0,0.313087,False,REGULAR,USD,call,2026-06-18,IBM


In [115]:
## hvol, filterdf, options 
# After filtering for open interest
options = options[options["openInterest"] > 100]

# Calculate average implied volatility for each symbol
result = options.groupby('symbol')['impliedVolatility'].mean().reset_index()

print(result)


    symbol  impliedVolatility
0       AA           0.517241
1      AAL           0.479701
2     AAPL           0.313554
3     ABNB           0.404486
4      ADT           0.383063
..     ...                ...
223   XPEV           0.600071
224    XRX           0.698001
225    YMM           0.495366
226   ZETA           0.675833
227    ZIM           0.855554

[228 rows x 2 columns]


In [ ]:
avg_iv = options.groupby('symbol')['impliedVolatility'].mean().reset_index()
avg_iv.columns = ['symbol', 'avg_iv']  # Rename for clarity
filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()
avg_iv['symbol'] = avg_iv['symbol'].astype(str).str.upper()
filterdf = filterdf.merge(avg_iv, on='symbol', how='left')

filterdf = filterdf.rename(columns={'hvol': 'historical_volatility', 'avg_iv': 'average_implied_volatility'})
filterdf["iv/hv_ratio"] = filterdf["average_implied_volatility"] / filterdf["historical_volatility"]

import datetime

today = datetime.datetime.today().strftime('%Y-%m-%d')
filename = f'/Users/nshaffer/Desktop/equity_vol_screen_{today}.csv'

filterdf.to_csv(filename, index=False)


filterdf

ValueError: cannot reindex on an axis with duplicate labels